# Gemaakt op 19/05/2025 door martijn

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats as st
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
# the imported functions

### Portfolio assignment 22
30 min: Let Yolov8 do predictions on your own choosen dataset (look for a nice images dataset on Roboflow or Kaggle or....).
- Load your data in and have a quick look
- Let YOLO predict from one of your images the objects
- Describe if the predictions are correct according to you, just by looking at the images.


ik heb hier een car [hand gesture dataset](https://www.kaggle.com/datasets/adhoppin/hand-gestures-dataset).

ik gok dat hij hierna niet goed zal werken op de foto's die ik hem ga geven. Maar ik denk dat hij wel goed zal werken op de supplied test dataset

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Get the current directory
current_dir = os.getcwd()

# Construct the base path dynamically
base_path = os.path.join(current_dir, 'train')

# Get the first image file in the folder
image_files = os.listdir(base_path)  # List all files in the folder

print("Image files in the folder:", len(image_files))  # Print the list of image files
first_image_path = os.path.join(base_path, image_files[0])  # Get the path of the first image

# Read the first image
image = cv2.imread(first_image_path)

# Convert from BGR (OpenCV) to RGB (matplotlib)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the first image
plt.imshow(image_rgb)
plt.axis('off')  # Hide axes for a cleaner look
plt.show()

nu hebbrn er de 1e foto gepakt :3

peace out ;)



In [ ]:
from sklearn.utils import shuffle
import pandas as pd

#Load the labels which are vectors with a reference to the image
annotations = pd.read_csv('train/_annotations.csv')
annotations = shuffle(annotations, random_state=34)
annotations.head()

In [ ]:
print(annotations.shape)

In [ ]:
classes = annotations['class'].unique()
print(classes)

er zijn dus 14 verschillende gestures

In [ ]:
annotations['class'].value_counts()

In [ ]:
from matplotlib import image
for c in classes:
    image_name = annotations[annotations['class'] == c].iloc[0,0]
    gesture = image.imread(os.path.join('./train/', image_name))
    plt.figure()
    plt.imshow(gesture)
    plt.title(f"Example of class {c}")
    plt.show()

okay so now i will give them unofficial class Names

In [ ]:
from matplotlib import image
names = {
    "0": "Peace",
    "1": "Walking",
    "2": "Double Flat Hands",
    "3": "Tower",
    "4": "Pacman?",
    "5": "Upwards Fist",
    "6": "Me",
    "7": "Point at Hand",
    "8": "Point at Walking",
    "9": "Magic Hands",
    "10": "Fingerguns",
    "11": "Flat Hand",
    "12": "Chin Hold Point",
    "13": "No Money",
}
classes.sort()
for c in classes:
    image_name = annotations[annotations['class'] == c].iloc[0,0]
    image = cv2.imread(os.path.join('./train/', image_name))
    image_rgb = image
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the first image
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.title(f"Example of class {c}, {names[str(c)]}")
    plt.show()

I made these calssnames myself. I had to improvice a little because the inverted colors make it a little bit harder to see  (the inverted colors is only an issue on pycharm)

In [ ]:
labels = {
    0: "Peace",
    1: "Walking",
    2: "Double Flat Hands",
    3: "Tower",
    4: "Pacman?",
    5: "Upwards Fist",
    6: "Me",
    7: "Point at Hand",
    8: "Point at Walking",
    9: "Magic Hands",
    10: "Fingerguns",
    11: "Flat Hand",
    12: "Chin Hold Point",
    13: "No Money",
}

I was too lazy to convert the above example yo be using intagers as `key`

In [ ]:
import matplotlib
# matplotlib.use('TkAgg')
boxes = {}
images = {}

print(annotations.head())

for class_id in classes:
    # get all the rows for the current class
    print(f"Processing class {class_id}...")
    target_rows = annotations[annotations['class'] == class_id]
    print(f"Number of rows for class {class_id}: {len(target_rows)}")
    # Check if there are any rows for the current class. If not, skip to the next class
    if target_rows.empty:
        print(f"No data found for class {class_id}. Skipping.")
        continue

    # Search for first image on disk from the dataframe
    image_found = False
    for _, row in target_rows.iterrows():
        file_name = row['filename']
        image_path = os.path.join(base_path, file_name)

        if not os.path.exists(image_path):
            continue

        print(f"Found image {image_path} for class {class_id}.")

        # Lees de afbeelding
        image = matplotlib.pyplot.imread(image_path)

        if image is None:
            print(f"Failed to load image {image_path}. Trying next row...")
            continue
        else:
            image_found = True
            break

    if image_found:
        print(f"Image found AND read for class {class_id}: {image_path}")
        # Convert from BGR (OpenCV) to RGB (matplotlib)
        image_rgb =image

        images[class_id] = image_rgb
        boxes[class_id] = [
            int(row['xmin']),
            int(row['xmax']),
            int(row['ymin']),
            int(row['ymax'])
        ]



In [ ]:
## Let's visualise the 5 different classes that exists with their bounding boxes in a image
for class_id in classes:
    xmin, xmax, ymin, ymax = boxes[class_id][0], boxes[class_id][1], boxes[class_id][2], boxes[class_id][3]
    # img = cv2.cvtColor(images[class_id], cv2.COLOR_BGR2RGB)
    img = images[class_id]
    # print(images[class_id].shape)

    plt.figure(figsize=(4, 6))
    plt.title("Label " + labels[class_id])
    plt.imshow(img)
    plt.gca().add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color='yellow', fill=False, linewidth=2))

    plt.show()

In [ ]:
from ultralytics import YOLO
import PIL
from PIL import Image
from IPython.display import display
import os
import pathlib


In [ ]:
# Load the YOLO v8 model
model = YOLO("yolov8m.pt")

In [ ]:
import numpy as np

np.random.seed(80)
random_num=np.random.randint(0, 100)
example = os.path.join(base_path, image_files[random_num])  # Get the path of the first image

In [ ]:
results = model.predict(source=example, save=True, conf=0.2, iou=0.2) # change conf and iou

In [ ]:
# Return the information (class, coordinates of the box and the confidence) of each predicted object
result = results[0]
for box in result.boxes:
    class_id = result.names[box.cls[0].item()]
    cords = box.xyxy[0].tolist()
    cords = [round(x) for x in cords]
    conf = round(box.conf[0].item(), 2)
    print("Object type:", class_id)
    print("Coordinates:", cords)
    print("Probability:", conf)
    print("---")

In [ ]:
# Plotting the predicted results
plot = results[0].plot()

# Convert the plot from BGR to RGB
plot = cv2.cvtColor(plot, cv2.COLOR_BGR2RGB)

# Create a larger figure size
plt.figure(figsize=(10, 10))
plt.imshow(plot)
plt.axis('off')
plt.show()

yeah that is not accurate xD

In [ ]:
filename = os.path.basename(example)


In [ ]:
annotations[annotations['filename']==filename]

In [ ]:
# Find all matching rows in annotations for this filename
matching_rows = annotations[annotations['filename'] == filename]

# Construct the full image path
image_path = os.path.join(base_path, filename)

# Read and convert the image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for Matplotlib

# Plot the image
plt.figure(figsize=(10, 10))
plt.imshow(image)

for index, row in matching_rows.iterrows():
    xmin, xmax, ymin, ymax = row['xmin'], row['xmax'], row['ymin'], row['ymax']

    plt.gca().add_patch(
        plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                      edgecolor='yellow', fill=False, linewidth=2)
    )

    # Add the index number above the bounding box
    plt.text(xmin, ymin - 5, f"{index}", color='yellow',
             fontsize=12, backgroundcolor='none')

plt.axis('off')  # Hide axes for a cl
plt.show()

In [ ]:
import pandas as pd

# Define the mapping from YOLO class IDs to the new custom label IDs
yolo_to_custom = {
    2: 1,  # Car -> Class ID 2 maps to 1
    7: 2,  # Truck -> Class ID 7 maps to 2
    0: 3,  # Person -> Class ID 0 maps to 3
    1: 4,  # Bicycle -> Class ID 1 maps to 4
    9: 5   # Traffic light -> Class ID 9 maps to 5
}

# Initialize a list to hold the rows of the DataFrame
predictions_data = []

# Example: Extract YOLO predictions and ground truth for each frame (this should come from results[0])
for i, box in enumerate(results[0].boxes):  # Iterate through each prediction
    frame = filename  # You would retrieve the frame name here

    # Extract xywh (center_x, center_y, width, height)
    center_x, center_y, width, height = box.xywh[0].int().tolist()  # Convert to integers

    # Convert to xmin, ymin, xmax, ymax
    xmin = int(center_x - width / 2)
    ymin = int(center_y - height / 2)
    xmax = int(center_x + width / 2)
    ymax = int(center_y + height / 2)

    # Convert class_id tensor to integer
    class_id = box.cls.int().item()  # Convert class tensor to integer

    # Map YOLO class_id to the custom label ID
    custom_class_id = yolo_to_custom.get(class_id, -1)  # Get the custom label ID (-1 for unknown)

    # Append the prediction data to the list, now without the class_label
    predictions_data.append([frame, xmin, xmax, ymin, ymax, custom_class_id])

# Convert the predictions data to a pandas DataFrame
predictions_df = pd.DataFrame(predictions_data, columns=['frame', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'])

# Show the resulting DataFrame
predictions_df

In [ ]:
# Function to calculate IoU
def compute_iou(pred_box, gt_box):
    x1, y1, x2, y2 = max(pred_box[0], gt_box[0]), max(pred_box[1], gt_box[1]), min(pred_box[2], gt_box[2]), min(pred_box[3], gt_box[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    pred_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
    gt_area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
    union_area = pred_area + gt_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

ground_truth_df = annotations[annotations['filename'] == filename]

# List to hold indices, ground truth, and IoU values
iou_info = []
# Calculate IoU without applying a threshold
for i, pred in predictions_df.iterrows():
    frame = pred['frame']
    pred_box = [pred['xmin'], pred['ymin'], pred['xmax'], pred['ymax']]
    pred_class = pred['class_id']

    # Find the corresponding ground truth
    gt_boxes = ground_truth_df[ground_truth_df['filename'] == frame]

    for j, gt in gt_boxes.iterrows():
        gt_box = [gt['xmin'], gt['ymin'], gt['xmax'], gt['ymax']]
        gt_class = gt['class']

        # Calculate IoU between prediction and ground truth
        iou = compute_iou(pred_box, gt_box)

        # Store the index, ground truth, and IoU for all predictions
        iou_info.append([i, j, pred_class, gt_class, iou])

# Create DataFrame to show the IoU results with prediction and ground truth indices
iou_df = pd.DataFrame(iou_info, columns=['prediction_index', 'ground_truth_index', 'pred_class_id', 'gt_class_id', 'iou'])

# Now, we can calculate the confusion matrix for all predictions (no threshold)
predicted_classes = iou_df['pred_class_id'].values
actual_classes = iou_df['gt_class_id'].values

# Compute confusion matrix
cm = confusion_matrix(actual_classes, predicted_classes)

# Visualize the confusion matrix using Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Car', 'Truck', 'Person', 'Bicycle', 'Traffic Light'], yticklabels=['Car', 'Truck', 'Person', 'Bicycle', 'Traffic Light'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (All Predictions)')
plt.show()

# Output the IoU values for all predictions
print("\nIoU per object:")
print(iou_df[['prediction_index', 'ground_truth_index', 'iou']])

Okay hij is dus 2 keer goed op img 1, en 1 keer fout op img 1. Laten we nu de hele test dataset nemen

In [ ]:
#Load the labels which are vectors with a reference to the image
annotations = pd.read_csv('valid/_annotations.csv')
annotations = shuffle(annotations, random_state=34)
annotations.head()

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from ultralytics import YOLO

# 1. Load validation annotations
ann = pd.read_csv('valid/_annotations.csv')
# we'll assume every row is one object; if you have multiple per image, that's fine

# 2. YOLO model
model = YOLO("yolov8m.pt")

# 3. Mapping from YOLO‐class → your custom class_id
yolo_to_custom = {
    2: 1,   # Car → Peace
    7: 2,   # Truck → Walking
    0: 3,   # Person → Double Flat Hands
    1: 4,   # Bicycle → Tower
    9: 5    # Traffic light → Pacman?
}
# (adjust to your real mapping)

# 4. Prepare lists
y_true = []
y_pred = []

# 5. Loop over each unique image in valid/
for fname in ann['filename'].unique():
    # run YOLO once per image
    res = model.predict(source=os.path.join('valid', fname), conf=0.2, iou=0.2)[0]

    # get all GT for this image
    gt_rows = ann[ann['filename'] == fname]
    # collect all GT class IDs
    gt_classes = gt_rows['class'].tolist()

    # for each predicted box:
    for b in res.boxes:
        pred_yolo_id = int(b.cls[0].item())
        pred_custom = yolo_to_custom.get(pred_yolo_id, -1)
        # Here you could match by IoU to a specific GT box
        # but for a simple overall confusion we'll just pair every pred vs. every GT
        # (or pick the highest‐IoU GT if you want one‐to‐one)
        for gt in gt_classes:
            y_true.append(gt)
            y_pred.append(pred_custom)

# 6. Compute confusion matrix
labels = sorted(set(y_true) | set(y_pred))
cm = confusion_matrix(y_true, y_pred, labels=labels)

# 7. Plot it
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d',
            xticklabels=labels, yticklabels=labels,
            cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Validation Confusion Matrix')
plt.show()


hmmm, dat ziet er niet al te best uit xDD


nu heb ik zin om random fotos er doorheen te gooien xD

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# 1. Load your YOLOv8 model once
model = YOLO("yolov8m.pt")

# 2. Path to your folder of images
me_dir = "me"
# adjust extensions if you have other types
image_files = [
    f for f in os.listdir(me_dir)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
]

# 3. Loop through each image, run predict, and plot
for img_name in image_files:
    img_path = os.path.join(me_dir, img_name)

    # run inference (you can tweak conf and iou thresholds)
    results = model.predict(source=img_path, conf=0.2, iou=0.2)
    res = results[0]

    # get the plotted image (with boxes & labels)
    plot = res.plot()
    # convert from BGR (OpenCV) to RGB (matplotlib)
    plot = cv2.cvtColor(plot, cv2.COLOR_BGR2RGB)

    # display
    plt.figure(figsize=(10, 10))
    plt.imshow(plot)
    plt.title(f"Predictions for {img_name}")
    plt.axis("off")
    plt.show()


## findings

over all i see that it recognises people quite accurately. And so does it recognise ducks.

BUt the hand gestures, or items, it is not the best at it.

still impressiver though